# TPS 12 - Feature Engineering

In [1]:
# Global variables for testing changes to this notebook quickly
RANDOM_SEED = 0
NUM_FOLDS = 6
TRAIN_SIZE = 500000

In [2]:
import numpy as np
import pandas as pd
import time
import os
import pyarrow
import gc

# Model/Evaluation
from sklearn.base import clone
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, recall_score
from sklearn.inspection import permutation_importance
from catboost import CatBoostClassifier

# Hide warnings
import warnings
warnings.filterwarnings('ignore')

# Load and Prepare Data

In [3]:
# Load full training data
train = pd.read_feather('../data/train.feather')

# Drop low/no variance 
train.drop(["Soil_Type7", "Id", "Soil_Type15"], axis=1, inplace=True)
train = train[train.Cover_Type != 5]

# Label Encoding
new_encoder = LabelEncoder()
train["Cover_Type"] = new_encoder.fit_transform(train["Cover_Type"])

# Split synthetic data
train, test = train_test_split(
    train, 
    train_size = TRAIN_SIZE, 
    random_state = RANDOM_SEED,
    stratify = train['Cover_Type'],
)
y_train = train['Cover_Type']


# features, data structure for summary scores
features = [x for x in train.columns if x not in ['Id','Cover_Type']]
nonsoil = [x for x in features if not x.startswith('Soil_Type')]
new_rows = list()
gc.collect()

print(f'Training Size: {train.shape[0]} rows, {train.shape[1]} cols')
print(f'Holdout Size: {test.shape[0]} rows, {test.shape[1]} cols\n')

Training Size: 500000 rows, 53 cols
Holdout Size: 3499999 rows, 53 cols



# Model 

In [4]:
# LightGBM Classifier
catboost_pipeline = make_pipeline(
    CatBoostClassifier(
        eval_metric = 'MultiClass',
        boosting_type = 'Plain',
        verbose = False,
        random_state = RANDOM_SEED,
    ),
)

# Scoring Function

In [5]:
def score_features(sklearn_model, processing = None):
    
    # Original Training/Test Split
    features = [x for x in train.columns if x not in ['Id','Cover_Type']]
    X_temp, X_test = train[features], test[features]
    y_temp, y_test = train['Cover_Type'], test['Cover_Type']
    
    # Feature Engineering
    if processing:
        X_temp = processing(X_temp)
        X_test = processing(X_test)
    
    # Store the out-of-fold predictions
    test_preds = np.zeros((X_test.shape[0],6))
    oof_preds = np.zeros((X_temp.shape[0],))
    fi_scores = np.zeros((X_temp.shape[1],))
    scores, times = np.zeros(NUM_FOLDS), np.zeros(NUM_FOLDS)
    
    # Stratified k-fold cross-validation
    skf = StratifiedKFold(n_splits = NUM_FOLDS, shuffle = True, random_state = RANDOM_SEED)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(train[features],train['Cover_Type'])):
       
        # Training and Validation Sets
        X_train, X_valid = X_temp.iloc[train_idx], X_temp.iloc[valid_idx]
        y_train, y_valid = y_temp.iloc[train_idx], y_temp.iloc[valid_idx]
        
        # Create model
        start = time.time()
        model = clone(sklearn_model)
        model.fit(X_train, y_train)

        # Permutation Importance
        result = permutation_importance(
            model, X_valid, y_valid, 
            random_state=RANDOM_SEED
        )
        fi_scores += result.importances_mean / NUM_FOLDS

        # validation/holdout predictions
        valid_preds = np.ravel(model.predict(X_valid))
        oof_preds[valid_idx] = valid_preds
        test_preds += model.predict_proba(X_test)

        # Save scores and times
        scores[fold] = accuracy_score(y_valid, valid_preds)
        end = time.time()
        times[fold] = end-start
        print(f'Fold {fold} Accuracy:  {round(scores[fold], 5)} in {round(end-start,2)}s.')
        time.sleep(0.5)
    
    features = [x for x in X_temp.columns]
    nonsoil = [x for x in X_test.columns if not x.startswith('Soil_Type')]
    test_preds = np.argmax(test_preds, axis = 1)
    test_score = accuracy_score(y_test, test_preds)
    #print('\n'+model.__class__.__name__)
    print("Train Accuracy:", round(scores.mean(), 5))
    print('Test Accuracy:', round(test_score, 5))
    print(f'Training Time: {round(times.sum(), 2)}s')
    
    fi_scores = pd.Series(
        data = fi_scores, 
        index = features
    ).loc[nonsoil].sort_values()
    
    return scores.mean(), oof_preds, test_score, fi_scores

# Baseline

In [6]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    catboost_pipeline
)

new_rows.append((
    'Baseline', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95864 in 196.05s.
Fold 1 Accuracy:  0.95769 in 199.69s.
Fold 2 Accuracy:  0.95661 in 189.12s.
Fold 3 Accuracy:  0.95824 in 186.47s.
Fold 4 Accuracy:  0.95796 in 190.46s.
Fold 5 Accuracy:  0.95712 in 194.44s.
Train Accuracy: 0.95771
Test Accuracy: 0.95828
Training Time: 1156.23s


Hillshade_9am                        -0.000034
Slope                                -0.000034
Hillshade_3pm                         0.000035
Wilderness_Area2                      0.000040
Aspect                                0.000054
Hillshade_Noon                        0.000333
Wilderness_Area4                      0.002174
Wilderness_Area1                      0.005410
Horizontal_Distance_To_Hydrology      0.010608
Vertical_Distance_To_Hydrology        0.018212
Wilderness_Area3                      0.023643
Horizontal_Distance_To_Fire_Points    0.031228
Horizontal_Distance_To_Roadways       0.047768
Elevation                             0.461603
dtype: float64

# Feature Engineering


1. Aspect Features
2. Hillshade Features
3. Water Features
4. Count Features
5. Water/Fire Interactions
6. Roadway Interactions
7. Elevation Interactions

## 1. Aspect Features

In [7]:
def aspect_features(data):
    df = data.copy()
    df['Aspect_360'] = df['Aspect'] % 360
    df['Aspect_Sine'] = (df['Aspect']* np.pi / 180).apply(np.sin)
    df['Aspect_Alt'] = (df['Aspect']-180).where(
        df['Aspect']+180 > 360, df['Aspect'] + 180
    )
    return df

In [8]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    catboost_pipeline, 
    aspect_features
)

new_rows.append((
    'Aspect_Features', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95854 in 156.2s.
Fold 1 Accuracy:  0.9575 in 158.25s.
Fold 2 Accuracy:  0.95693 in 153.93s.
Fold 3 Accuracy:  0.95817 in 155.72s.
Fold 4 Accuracy:  0.95792 in 155.62s.
Fold 5 Accuracy:  0.95716 in 155.3s.
Train Accuracy: 0.9577
Test Accuracy: 0.95817
Training Time: 935.02s


Aspect_Alt                           -0.000036
Aspect_Sine                          -0.000020
Slope                                -0.000012
Hillshade_3pm                         0.000006
Aspect_360                            0.000041
Hillshade_9am                         0.000060
Wilderness_Area2                      0.000066
Aspect                                0.000070
Hillshade_Noon                        0.000371
Wilderness_Area4                      0.001853
Wilderness_Area1                      0.005905
Horizontal_Distance_To_Hydrology      0.010753
Vertical_Distance_To_Hydrology        0.018298
Wilderness_Area3                      0.023145
Horizontal_Distance_To_Fire_Points    0.031260
Horizontal_Distance_To_Roadways       0.047864
Elevation                             0.461542
dtype: float64

## 2. Hillshade Features

In [9]:
def hillshade_features(data):
    df = data.copy()
    shade_features = ['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']

    # Clip Range
    df["Hillshade_9am_Clipped"] = df["Hillshade_9am"].clip(lower=0, upper=255)
    df["Hillshade_Noon_Clipped"] = df["Hillshade_9am"].clip(lower=0, upper=255)
    df["Hillshade_3pm_Clipped"] = df["Hillshade_9am"].clip(lower=0, upper=255)
    
    # Hillshade
    #df["Hillshade_Avg"] = df[shade_features].mean(axis=1)
    df["Hillshade_Sum"] = df[shade_features].sum(axis=1)
    df['Hillshade_Range'] = df[shade_features].max(axis=1) - df[shade_features].min(axis=1)
    
    return df

In [10]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    catboost_pipeline, 
    hillshade_features
)

new_rows.append((
    'Hillshade_Features', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95848 in 159.81s.
Fold 1 Accuracy:  0.95774 in 160.81s.
Fold 2 Accuracy:  0.95682 in 159.45s.
Fold 3 Accuracy:  0.95822 in 160.05s.
Fold 4 Accuracy:  0.95792 in 159.74s.
Fold 5 Accuracy:  0.95715 in 159.21s.
Train Accuracy: 0.95772
Test Accuracy: 0.95822
Training Time: 959.06s


Hillshade_3pm                        -1.719997e-05
Hillshade_Range                      -1.359999e-05
Slope                                -1.280011e-05
Hillshade_Sum                        -9.599774e-06
Hillshade_9am_Clipped                -8.001088e-07
Hillshade_Noon_Clipped               -8.000032e-07
Hillshade_3pm_Clipped                 3.600024e-06
Hillshade_9am                         3.680027e-05
Wilderness_Area2                      4.319965e-05
Aspect                                6.679954e-05
Hillshade_Noon                        3.860001e-04
Wilderness_Area4                      2.549201e-03
Wilderness_Area1                      5.769201e-03
Horizontal_Distance_To_Hydrology      1.061160e-02
Vertical_Distance_To_Hydrology        1.829160e-02
Wilderness_Area3                      2.347880e-02
Horizontal_Distance_To_Fire_Points    3.116560e-02
Horizontal_Distance_To_Roadways       4.777960e-02
Elevation                             4.614904e-01
dtype: float64

## 3. Water Features

In [11]:
# Helper function
def start_at_eps(series, eps=1e-10): 
    return series - series.min() + eps

def water_features(data):
    df = data.copy()
    
    # use float64 for squaring
    df["Horizontal_Distance_To_Hydrology"] = df["Horizontal_Distance_To_Hydrology"].astype('float64')
    df["Vertical_Distance_To_Hydrology"] = df["Vertical_Distance_To_Hydrology"].astype('float64')
    pos_h_hydrology = start_at_eps(df["Horizontal_Distance_To_Hydrology"])
    pos_v_hydrology = start_at_eps(df['Vertical_Distance_To_Hydrology'])
    
    # Manhatten Distances
    df["Hydro_Taxicab"] = np.abs(df["Horizontal_Distance_To_Hydrology"]) + np.abs(df["Vertical_Distance_To_Hydrology"])
    df['Hydro_Taxicab_Pos'] = (pos_h_hydrology ** 2 + pos_v_hydrology ** 2).apply(np.sqrt).rename('Euclidean_positive_hydrology').astype(np.float32)
    
    # Euclidean Distance
    df["Hydro_Euclid"] = (df["Horizontal_Distance_To_Hydrology"]**2 + np.abs(df["Vertical_Distance_To_Hydrology"])**2)**0.5
    df['Hydro_Euclid_Pos'] = (pos_h_hydrology ** 2 + pos_v_hydrology ** 2).apply(np.sqrt)
    
    # Misc Features
    df['Water_Direction'] = df['Vertical_Distance_To_Hydrology'].apply(np.sign)
    df['Water Elevation'] = df['Elevation'] - df['Vertical_Distance_To_Hydrology']
    
    # Store each as float32
    df["Horizontal_Distance_To_Hydrology"] = df["Horizontal_Distance_To_Hydrology"].astype('float32')
    df["Vertical_Distance_To_Hydrology"] = df["Vertical_Distance_To_Hydrology"].astype('float32')
    df["Hydro_Taxicab"] = df["Hydro_Taxicab"].astype('float32')
    df['Hydro_Taxicab_Pos'] = df['Hydro_Taxicab_Pos'].astype('float32')
    df["Hydro_Euclid"] = df["Hydro_Euclid"].astype('float32')
    df['Hydro_Euclid_Pos'] = df['Hydro_Euclid_Pos'].astype('float32')
    df['Water_Direction'] = df['Water_Direction'].astype('float32')
    df['Water Elevation'] = df['Water Elevation'].astype('float32')
    
    return df

In [12]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    catboost_pipeline, 
    water_features
)

new_rows.append((
    'Water_Features', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95907 in 162.79s.
Fold 1 Accuracy:  0.95812 in 162.18s.
Fold 2 Accuracy:  0.95806 in 162.31s.
Fold 3 Accuracy:  0.95912 in 161.71s.
Fold 4 Accuracy:  0.95901 in 162.15s.
Fold 5 Accuracy:  0.95856 in 161.68s.
Train Accuracy: 0.95866
Test Accuracy: 0.95907
Training Time: 972.83s


Hillshade_3pm                        -0.000044
Slope                                -0.000012
Water_Direction                       0.000003
Hillshade_9am                         0.000041
Aspect                                0.000056
Wilderness_Area2                      0.000060
Vertical_Distance_To_Hydrology        0.000192
Hydro_Euclid_Pos                      0.000236
Hydro_Taxicab_Pos                     0.000278
Hillshade_Noon                        0.000406
Hydro_Taxicab                         0.000612
Hydro_Euclid                          0.001254
Horizontal_Distance_To_Hydrology      0.001412
Wilderness_Area4                      0.002373
Wilderness_Area1                      0.005886
Wilderness_Area3                      0.023550
Horizontal_Distance_To_Fire_Points    0.031695
Horizontal_Distance_To_Roadways       0.047802
Water Elevation                       0.156168
Elevation                             0.311370
dtype: float64

## 4. Count Features

In [13]:
def count_features(data):
    
    df = data.copy()
    soil_features = [x for x in df.columns if x.startswith("Soil_Type")]
    wilderness_features = [x for x in df.columns if x.startswith("Wilderness_Area")]

    # Count features
    df["Soil_Count"] = df[soil_features].apply(sum, axis=1)
    df["Wilderness_Count"] = df[wilderness_features].apply(sum, axis=1)
    
    return df

In [14]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    catboost_pipeline, 
    count_features
)

new_rows.append((
    'Count_Features', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.96 in 198.24s.
Fold 1 Accuracy:  0.95915 in 203.84s.
Fold 2 Accuracy:  0.9589 in 202.96s.
Fold 3 Accuracy:  0.95949 in 198.45s.
Fold 4 Accuracy:  0.95968 in 196.06s.
Fold 5 Accuracy:  0.95837 in 197.31s.
Train Accuracy: 0.95927
Test Accuracy: 0.95967
Training Time: 1196.86s


Hillshade_9am                        -0.000066
Slope                                -0.000056
Wilderness_Count                      0.000020
Hillshade_3pm                         0.000040
Aspect                                0.000097
Wilderness_Area2                      0.000101
Hillshade_Noon                        0.000415
Wilderness_Area4                      0.001306
Wilderness_Area1                      0.004042
Horizontal_Distance_To_Hydrology      0.010873
Vertical_Distance_To_Hydrology        0.018375
Wilderness_Area3                      0.024911
Horizontal_Distance_To_Fire_Points    0.032024
Soil_Count                            0.035319
Horizontal_Distance_To_Roadways       0.048739
Elevation                             0.462869
dtype: float64

## 5. Water/Fire Interactions

In [15]:
def hydrofire_interactions(data):
    
    df = data.copy()
    df['Hydro_Fire_Sum'] = df['Horizontal_Distance_To_Hydrology'] + df['Horizontal_Distance_To_Fire_Points']
    df['Hydro_Fire_AbsDiff'] = abs(df['Horizontal_Distance_To_Hydrology'] - df['Horizontal_Distance_To_Fire_Points'])
    df['Hydro_Fire_EpsSum'] = start_at_eps(df['Horizontal_Distance_To_Hydrology']) + start_at_eps(df['Horizontal_Distance_To_Fire_Points'])
    df['Hydro_Fire_Diff'] = df['Horizontal_Distance_To_Hydrology'] - df['Horizontal_Distance_To_Fire_Points']
    return df

In [16]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    catboost_pipeline, 
    hydrofire_interactions
)

new_rows.append((
    'Water_Fire', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95889 in 159.13s.
Fold 1 Accuracy:  0.95759 in 159.31s.
Fold 2 Accuracy:  0.95733 in 158.76s.
Fold 3 Accuracy:  0.95813 in 159.05s.
Fold 4 Accuracy:  0.95889 in 158.75s.
Fold 5 Accuracy:  0.95772 in 159.71s.
Train Accuracy: 0.95809
Test Accuracy: 0.95839
Training Time: 954.71s


Hillshade_3pm                        -0.000002
Hillshade_9am                         0.000010
Slope                                 0.000032
Wilderness_Area2                      0.000073
Aspect                                0.000114
Hillshade_Noon                        0.000494
Hydro_Fire_EpsSum                     0.001792
Wilderness_Area4                      0.001819
Hydro_Fire_Diff                       0.002858
Hydro_Fire_AbsDiff                    0.003080
Hydro_Fire_Sum                        0.003208
Horizontal_Distance_To_Fire_Points    0.004446
Wilderness_Area1                      0.006311
Horizontal_Distance_To_Hydrology      0.008095
Vertical_Distance_To_Hydrology        0.018412
Wilderness_Area3                      0.023100
Horizontal_Distance_To_Roadways       0.048192
Elevation                             0.461953
dtype: float64

## 6. Roadway Interactions

In [17]:
def roadway_interactions(data):
    df = data.copy()
    df['Hydro_Road_1'] = abs(df['Horizontal_Distance_To_Hydrology'] + df['Horizontal_Distance_To_Roadways'])
    df['Hydro_Road_2'] = abs(df['Horizontal_Distance_To_Hydrology'] - df['Horizontal_Distance_To_Roadways'])
    df['Fire_Road_1'] = abs(df['Horizontal_Distance_To_Fire_Points'] + df['Horizontal_Distance_To_Roadways'])
    df['Fire_Road_2'] = abs(df['Horizontal_Distance_To_Fire_Points'] - df['Horizontal_Distance_To_Roadways'])
    return df

In [18]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    catboost_pipeline, 
    roadway_interactions
)

new_rows.append((
    'Road_Interactions', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95886 in 158.7s.
Fold 1 Accuracy:  0.95751 in 159.05s.
Fold 2 Accuracy:  0.95766 in 159.29s.
Fold 3 Accuracy:  0.9589 in 158.98s.
Fold 4 Accuracy:  0.95831 in 156.67s.
Fold 5 Accuracy:  0.95775 in 157.76s.
Train Accuracy: 0.95817
Test Accuracy: 0.95856
Training Time: 950.45s


Hillshade_3pm                        -0.000021
Slope                                 0.000018
Aspect                                0.000043
Hillshade_9am                         0.000044
Wilderness_Area2                      0.000082
Hillshade_Noon                        0.000470
Hydro_Road_1                          0.000854
Wilderness_Area4                      0.001574
Hydro_Road_2                          0.003722
Fire_Road_2                           0.004368
Wilderness_Area1                      0.005622
Fire_Road_1                           0.006018
Horizontal_Distance_To_Hydrology      0.009057
Horizontal_Distance_To_Fire_Points    0.017488
Vertical_Distance_To_Hydrology        0.018236
Wilderness_Area3                      0.023644
Horizontal_Distance_To_Roadways       0.025204
Elevation                             0.461998
dtype: float64

## 7. Elevation Interactions

In [19]:
def elevation_interactions(data):
    df = data.copy()
    df['Road_Elev_Int'] = df['Horizontal_Distance_To_Roadways'] * df['Elevation']
    df['VHydro_Elev_Int'] = df['Vertical_Distance_To_Hydrology'] * df['Elevation']
    df['Elev_VHydro_Diff'] = df.Elevation - df.Vertical_Distance_To_Hydrology
    df['Elev_HHydro_Diff'] = df.Elevation - df.Horizontal_Distance_To_Hydrology * 0.2
    
    return df

In [20]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    catboost_pipeline, 
    elevation_interactions
)

new_rows.append((
    'Elev_Interactions', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95915 in 157.93s.
Fold 1 Accuracy:  0.95844 in 158.75s.
Fold 2 Accuracy:  0.9575 in 158.37s.
Fold 3 Accuracy:  0.95928 in 158.16s.
Fold 4 Accuracy:  0.95895 in 157.56s.
Fold 5 Accuracy:  0.95836 in 156.67s.
Train Accuracy: 0.95861
Test Accuracy: 0.95902
Training Time: 947.44s


VHydro_Elev_Int                      -0.000035
Hillshade_3pm                        -0.000005
Road_Elev_Int                         0.000002
Slope                                 0.000012
Hillshade_9am                         0.000044
Wilderness_Area2                      0.000044
Aspect                                0.000059
Hillshade_Noon                        0.000402
Vertical_Distance_To_Hydrology        0.000447
Wilderness_Area4                      0.001786
Horizontal_Distance_To_Hydrology      0.004806
Wilderness_Area1                      0.005832
Wilderness_Area3                      0.023216
Horizontal_Distance_To_Fire_Points    0.031686
Elev_HHydro_Diff                      0.045442
Horizontal_Distance_To_Roadways       0.047697
Elevation                             0.143600
Elev_VHydro_Diff                      0.162484
dtype: float64

# Summary

In [21]:
pd.DataFrame.from_records(
    data = new_rows,
    columns = ['features','cv_scores','holdout','recall_0', 'recall_1','recall_2','recall_3','recall_4','recall_5']
).sort_values('holdout')

,features,cv_scores,holdout,recall_0,recall_1,recall_2,recall_3,recall_4,recall_5
1,Aspect_Features,0.957702,0.958166,0.964020,0.972249,0.875818,0.148936,0.417367,0.641655
2,Hillshade_Features,0.957720,0.958221,0.964298,0.972036,0.876553,0.127660,0.412465,0.642683
0,Baseline,0.957710,0.958279,0.964134,0.971895,0.877453,0.148936,0.420168,0.646666
5,Water_Fire,0.958092,0.958389,0.964516,0.972532,0.874346,0.127660,0.431373,0.646923
6,Road_Interactions,0.958166,0.958563,0.964674,0.972482,0.875327,0.127660,0.442577,0.644610
7,Elev_Interactions,0.958614,0.959017,0.964674,0.973207,0.876431,0.148936,0.431373,0.645509
3,Water_Features,0.958656,0.959074,0.965066,0.972935,0.876880,0.212766,0.436975,0.646023
4,Count_Features,0.959266,0.959671,0.964150,0.971891,0.884156,0.191489,0.466387,0.716562
